In [11]:
!pip install pandas numpy scikit-learn xgboost ipywidgets voila plotly pyarrow

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import plotly.express as px
import plotly.graph_objects as go
from itertools import cycle

# Generate synthetic dataset with 200 samples and 12 diseases
np.random.seed(42)
num_samples = 200

data = {
    'age': np.random.randint(18, 80, num_samples),
    'blood_pressure': np.random.randint(90, 180, num_samples),
    'cholesterol': np.random.randint(150, 300, num_samples),
    'bmi': np.round(np.random.uniform(18, 35, num_samples), 1),
    'glucose': np.random.randint(70, 200, num_samples),
    'exercise': np.random.randint(0, 7, num_samples),
    'smoking': np.random.choice([0, 1], num_samples),
    'alcohol': np.random.choice([0, 1, 2], num_samples, p=[0.7, 0.2, 0.1]),  # 0: none, 1: moderate, 2: heavy
    'stress': np.random.choice([0, 1, 2], num_samples, p=[0.5, 0.3, 0.2]),  # 0: low, 1: medium, 2: high
    'sleep': np.round(np.random.uniform(4, 9, num_samples), 1),
    'hypertension': np.random.choice([0, 1], num_samples, p=[0.7, 0.3]),
    'diabetes': np.random.choice([0, 1], num_samples, p=[0.8, 0.2]),
    'heart_disease': np.random.choice([0, 1], num_samples, p=[0.85, 0.15]),
    'arthritis': np.random.choice([0, 1], num_samples, p=[0.9, 0.1]),
    'copd': np.random.choice([0, 1], num_samples, p=[0.95, 0.05]),
    'asthma': np.random.choice([0, 1], num_samples, p=[0.9, 0.1]),
    'kidney_disease': np.random.choice([0, 1], num_samples, p=[0.95, 0.05]),
    'liver_disease': np.random.choice([0, 1], num_samples, p=[0.95, 0.05]),
    'osteoporosis': np.random.choice([0, 1], num_samples, p=[0.9, 0.1]),
    'depression': np.random.choice([0, 1], num_samples, p=[0.85, 0.15]),
    'anemia': np.random.choice([0, 1], num_samples, p=[0.9, 0.1]),
    'hyperthyroidism': np.random.choice([0, 1], num_samples, p=[0.95, 0.05]),
    'hypothyroidism': np.random.choice([0, 1], num_samples, p=[0.9, 0.1])
}

# Make some logical correlations
for i in range(num_samples):
    # Hypertension correlations
    if data['age'][i] > 50 and data['bmi'][i] > 25:
        data['hypertension'][i] = np.random.choice([0, 1], p=[0.4, 0.6])
    if data['cholesterol'][i] > 220:
        data['heart_disease'][i] = np.random.choice([0, 1], p=[0.6, 0.4])
    # Diabetes correlations
    if data['bmi'][i] > 30 and data['exercise'][i] < 2:
        data['diabetes'][i] = np.random.choice([0, 1], p=[0.5, 0.5])
    # Depression correlations
    if data['stress'][i] == 2 and data['sleep'][i] < 6:
        data['depression'][i] = np.random.choice([0, 1], p=[0.3, 0.7])

df = pd.DataFrame(data)

# Prepare features and targets
features = ['age', 'blood_pressure', 'cholesterol', 'bmi', 'glucose', 
            'exercise', 'smoking', 'alcohol', 'stress', 'sleep']
targets = ['hypertension', 'diabetes', 'heart_disease', 'arthritis', 'copd',
            'asthma', 'kidney_disease', 'liver_disease', 'osteoporosis',
            'depression', 'anemia', 'hyperthyroidism', 'hypothyroidism']

X = df[features]
y = df[targets]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model with class weighting
model = MultiOutputClassifier(
    RandomForestClassifier(
        n_estimators=100,
        class_weight='balanced',
        random_state=42
    )
)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=targets))

                 precision    recall  f1-score   support

   hypertension       0.60      0.17      0.26        18
       diabetes       1.00      0.09      0.17        11
  heart_disease       0.00      0.00      0.00         7
      arthritis       0.00      0.00      0.00         3
           copd       0.00      0.00      0.00         3
         asthma       0.00      0.00      0.00         3
 kidney_disease       0.00      0.00      0.00         3
  liver_disease       0.00      0.00      0.00         1
   osteoporosis       0.00      0.00      0.00         4
     depression       0.00      0.00      0.00        11
         anemia       0.00      0.00      0.00         5
hyperthyroidism       0.00      0.00      0.00         4
 hypothyroidism       0.00      0.00      0.00         4

      micro avg       0.67      0.05      0.10        77
      macro avg       0.12      0.02      0.03        77
   weighted avg       0.28      0.05      0.08        77
    samples avg       0.10   

C:\Users\asus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\asus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\asus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\asus\anaconda3\Lib\site-packages\sklearn\m

In [13]:
# Expanded recommendations for 12+ conditions
diet_plans = {
    'hypertension': {
        'name': 'Hypertension',
        'recommendations': [
            'DASH diet (Dietary Approaches to Stop Hypertension)',
            'Limit sodium to <2,300mg/day (ideally 1,500mg)',
            'Increase potassium-rich foods (bananas, spinach, sweet potatoes)',
            'Include calcium and magnesium rich foods',
            'Limit alcohol to 1 drink/day for women, 2 for men'
        ],
        'foods_to_avoid': ['Processed foods', 'Canned soups', 'Pickles', 'Soy sauce', 'Deli meats'],
        'beneficial_foods': ['Leafy greens', 'Berries', 'Beets', 'Oatmeal', 'Fatty fish', 'Garlic']
    },
    'diabetes': {
        'name': 'Diabetes',
        'recommendations': [
            'Monitor carbohydrate intake (focus on low glycemic index)',
            'Balance meals with protein, fat, and fiber',
            'Eat at regular intervals (3 meals + snacks if needed)',
            'Choose whole grains over refined carbohydrates',
            'Stay hydrated with water instead of sugary drinks'
        ],
        'foods_to_avoid': ['Sugary beverages', 'White bread', 'Pastries', 'Fried foods', 'Fruit juices'],
        'beneficial_foods': ['Non-starchy vegetables', 'Nuts and seeds', 'Legumes', 'Cinnamon', 'Apple cider vinegar']
    },
    'heart_disease': {
        'name': 'Heart Disease',
        'recommendations': [
            'Mediterranean diet pattern',
            'Increase omega-3 fatty acids (2+ servings fatty fish/week)',
            'Limit saturated fats (<7% of total calories)',
            'Increase soluble fiber (10-25g/day)',
            'Include plant sterols/stanols (2g/day)'
        ],
        'foods_to_avoid': ['Trans fats', 'Processed meats', 'Full-fat dairy', 'Fried foods', 'Packaged snacks'],
        'beneficial_foods': ['Salmon', 'Walnuts', 'Flaxseeds', 'Olive oil', 'Dark chocolate', 'Green tea']
    },
    'arthritis': {
        'name': 'Arthritis',
        'recommendations': [
            'Anti-inflammatory diet pattern',
            'Increase omega-3 fatty acids',
            'Include antioxidant-rich foods',
            'Consider nightshade vegetable elimination trial',
            'Maintain healthy weight to reduce joint stress'
        ],
        'foods_to_avoid': ['Processed sugars', 'Refined carbs', 'Fried foods', 'Excessive alcohol'],
        'beneficial_foods': ['Fatty fish', 'Turmeric', 'Ginger', 'Cherries', 'Broccoli', 'Green tea']
    },
    'copd': {
        'name': 'COPD',
        'recommendations': [
            'High-calorie, high-protein diet if underweight',
            'Small frequent meals if experiencing breathlessness',
            'Limit gas-producing foods if bloating is an issue',
            'Stay well hydrated to thin mucus secretions',
            'Ensure adequate vitamin D and calcium intake'
        ],
        'foods_to_avoid': ['Carbonated drinks', 'Excessive caffeine', 'High-salt foods', 'Dairy if increases mucus'],
        'beneficial_foods': ['Lean proteins', 'Complex carbs', 'Healthy fats', 'Antioxidant-rich fruits']
    },
    'asthma': {
        'name': 'Asthma',
        'recommendations': [
            'Anti-inflammatory diet pattern',
            'Increase magnesium-rich foods',
            'Include omega-3 fatty acids',
            'Consider vitamin D supplementation if deficient',
            'Identify and avoid food triggers if applicable'
        ],
        'foods_to_avoid': ['Sulfite-containing foods (dried fruits, wine)', 'Food allergens if sensitive'],
        'beneficial_foods': ['Apples', 'Bananas', 'Spinach', 'Salmon', 'Avocados', 'Ginger']
    },
    'kidney_disease': {
        'name': 'Kidney Disease',
        'recommendations': [
            'Control protein intake (0.6-0.8g/kg body weight)',
            'Limit phosphorus if levels are high',
            'Control potassium if levels are high',
            'Limit sodium to control blood pressure',
            'Maintain adequate calorie intake'
        ],
        'foods_to_avoid': ['Processed meats', 'Dairy if phosphorus restricted', 'Bananas/oranges if potassium restricted'],
        'beneficial_foods': ['Red bell peppers', 'Cabbage', 'Cauliflower', 'Garlic', 'Onions', 'Apples']
    },
    'liver_disease': {
        'name': 'Liver Disease',
        'recommendations': [
            'Moderate protein intake (1-1.2g/kg body weight)',
            'Include branched-chain amino acids',
            'Small frequent meals if appetite is poor',
            'Limit sodium if ascites present',
            'Avoid alcohol completely'
        ],
        'foods_to_avoid': ['Alcohol', 'High-fat foods', 'Raw shellfish', 'Excessive iron'],
        'beneficial_foods': ['Coffee', 'Oatmeal', 'Green tea', 'Berries', 'Grapes', 'Beetroot']
    },
    'osteoporosis': {
        'name': 'Osteoporosis',
        'recommendations': [
            'Ensure adequate calcium intake (1,000-1,200mg/day)',
            'Vitamin D (600-800 IU/day)',
            'Include magnesium and vitamin K rich foods',
            'Moderate protein intake',
            'Limit excessive sodium and caffeine'
        ],
        'foods_to_avoid': ['Excessive salt', 'Cola drinks', 'Alcohol in excess', 'Unbalanced high-protein diets'],
        'beneficial_foods': ['Dairy products', 'Leafy greens', 'Salmon', 'Almonds', 'Fortified foods']
    },
    'depression': {
        'name': 'Depression',
        'recommendations': [
            'Mediterranean-style diet pattern',
            'Increase omega-3 fatty acids',
            'Include probiotic-rich foods',
            'Ensure adequate B vitamins',
            'Limit processed foods and sugars'
        ],
        'foods_to_avoid': ['Processed foods', 'Sugary snacks', 'Excessive caffeine', 'Alcohol'],
        'beneficial_foods': ['Fatty fish', 'Fermented foods', 'Nuts and seeds', 'Dark chocolate', 'Turmeric']
    },
    'anemia': {
        'name': 'Anemia',
        'recommendations': [
            'Increase iron-rich foods (heme iron from meat more absorbable)',
            'Include vitamin C with iron sources to enhance absorption',
            'Ensure adequate folate and B12 intake',
            'Limit tea/coffee with meals as they inhibit iron absorption',
            'Consider cooking in cast iron pans'
        ],
        'foods_to_avoid': ['Tea/coffee with meals', 'Calcium supplements with iron-rich meals'],
        'beneficial_foods': ['Red meat', 'Organ meats', 'Shellfish', 'Beans', 'Dark leafy greens', 'Citrus fruits']
    },
    'hyperthyroidism': {
        'name': 'Hyperthyroidism',
        'recommendations': [
            'Ensure adequate calorie intake',
            'Increase calcium and vitamin D',
            'Small frequent meals if experiencing rapid metabolism',
            'Limit caffeine if experiencing anxiety',
            'Consider selenium-rich foods'
        ],
        'foods_to_avoid': ['Excessive iodine if advised by doctor', 'Caffeine if causing symptoms'],
        'beneficial_foods': ['Cruciferous vegetables', 'Berries', 'Brazil nuts', 'Dairy products', 'Leafy greens']
    },
    'hypothyroidism': {
        'name': 'Hypothyroidism',
        'recommendations': [
            'Ensure adequate iodine intake (unless autoimmune thyroiditis)',
            'Include selenium and zinc-rich foods',
            'Balance goitrogenic foods (cook cruciferous vegetables)',
            'Maintain fiber intake but separate from medication',
            'Stay hydrated'
        ],
        'foods_to_avoid': ['Raw cruciferous vegetables in large amounts', 'Soy in large amounts', 'Excessive fiber with medication'],
        'beneficial_foods': ['Brazil nuts', 'Fish', 'Dairy', 'Eggs', 'Berries', 'Yogurt']
    },
    'general_health': {
        'name': 'General Health',
        'recommendations': [
            'Eat a variety of colorful fruits and vegetables',
            'Choose whole grains over refined grains',
            'Include healthy fats (olive oil, nuts, avocados)',
            'Stay hydrated with water',
            'Practice mindful eating'
        ],
        'foods_to_avoid': ['Processed foods', 'Sugary snacks', 'Trans fats', 'Excessive alcohol'],
        'beneficial_foods': ['Whole foods', 'Lean proteins', 'Healthy fats', 'Complex carbohydrates']
    }
}

def get_combined_recommendations(predictions, conditions):
    """Generate combined recommendations for multiple conditions"""
    if not conditions:
        return diet_plans['general_health']
    
    combined = {
        'name': ' + '.join([diet_plans[c]['name'] for c in conditions]),
        'recommendations': [],
        'foods_to_avoid': [],
        'beneficial_foods': []
    }
    
    # Priority system for conflicting recommendations
    priority_order = [
        'kidney_disease', 'liver_disease', 'diabetes', 'heart_disease',
        'hypertension', 'copd', 'asthma', 'osteoporosis', 'arthritis',
        'hyperthyroidism', 'hypothyroidism', 'anemia', 'depression'
    ]
    
    # Sort conditions by priority
    conditions_sorted = sorted(conditions, key=lambda x: priority_order.index(x) if x in priority_order else len(priority_order))
    
    # Process each condition in order of priority
    for condition in conditions_sorted:
        plan = diet_plans[condition]
        
        # Add recommendations if not conflicting
        for rec in plan['recommendations']:
            if rec not in combined['recommendations']:
                combined['recommendations'].append(rec)
        
        # Handle food restrictions (most restrictive takes precedence)
        for food in plan['foods_to_avoid']:
            if food not in combined['foods_to_avoid']:
                combined['foods_to_avoid'].append(food)
        
        # Add beneficial foods
        for food in plan['beneficial_foods']:
            if food not in combined['beneficial_foods']:
                combined['beneficial_foods'].append(food)
    
    return combined

In [14]:
# Create comprehensive input widgets
style = {'description_width': '180px'}
layout = widgets.Layout(width='600px')

# Personal Information
personal_header = widgets.HTML("<h3>Personal Information</h3>")
age = widgets.IntSlider(value=35, min=18, max=100, description='Age:', style=style, layout=layout)
gender = widgets.RadioButtons(options=[('Male', 'male'), ('Female', 'female'), ('Other', 'other')], 
                            description='Gender:', style=style)

# Vital Signs
vitals_header = widgets.HTML("<h3>Vital Signs</h3>")
bp_systolic = widgets.IntSlider(value=120, min=80, max=200, description='Systolic BP:', style=style, layout=layout)
bp_diastolic = widgets.IntSlider(value=80, min=50, max=120, description='Diastolic BP:', style=style, layout=layout)
chol_total = widgets.IntSlider(value=180, min=100, max=300, description='Total Cholesterol:', style=style, layout=layout)
chol_hdl = widgets.IntSlider(value=50, min=20, max=100, description='HDL Cholesterol:', style=style, layout=layout)
bmi = widgets.FloatSlider(value=22.0, min=15, max=40, step=0.5, description='BMI:', style=style, layout=layout)
glucose = widgets.IntSlider(value=90, min=70, max=300, description='Glucose (mg/dL):', style=style, layout=layout)

# Lifestyle Factors
lifestyle_header = widgets.HTML("<h3>Lifestyle Factors</h3>")
exercise = widgets.IntSlider(value=3, min=0, max=10, description='Exercise (hrs/week):', style=style, layout=layout)
smoking = widgets.RadioButtons(options=[('Non-smoker', 0), ('Former smoker', 1), ('Current smoker', 2)], 
                             description='Smoking Status:', style=style)
alcohol = widgets.RadioButtons(options=[('None', 0), ('Moderate', 1), ('Heavy', 2)], 
                             description='Alcohol Consumption:', style=style)
stress = widgets.RadioButtons(options=[('Low', 0), ('Medium', 1), ('High', 2)], 
                            description='Stress Level:', style=style)
sleep = widgets.FloatSlider(value=7.0, min=4, max=10, step=0.5, description='Sleep (hrs/night):', style=style, layout=layout)

# Symptoms Checklist
symptoms_header = widgets.HTML("<h3>Symptoms Checklist</h3>")
symptoms = widgets.SelectMultiple(
    options=[
        'Fatigue', 'Frequent thirst', 'Frequent urination', 
        'Shortness of breath', 'Chest pain', 'Joint pain',
        'Persistent cough', 'Wheezing', 'Swelling in extremities',
        'Abdominal pain', 'Frequent headaches', 'Mood swings',
        'Unexplained weight loss', 'Unexplained weight gain',
        'Dizziness', 'Palpitations', 'Digestive issues'
    ],
    description='Current Symptoms:',
    disabled=False,
    style=style,
    layout=widgets.Layout(width='600px', height='150px')
)

predict_button = widgets.Button(description="Analyze Health Status", 
                              button_style='success',
                              layout=widgets.Layout(width='200px', height='50px'))
output = widgets.Output()

# Create tabs for different sections
tab = widgets.Tab()
health_summary = widgets.Output()
detailed_reports = widgets.Output()
diet_plan = widgets.Output()
lifestyle_plan = widgets.Output()
visualization = widgets.Output()

tab.children = [health_summary, detailed_reports, diet_plan, lifestyle_plan, visualization]
tab.titles = ['Summary', 'Detailed Report', 'Diet Plan', 'Lifestyle Plan', 'Visualization']

# Display widgets
input_box = widgets.VBox([
    personal_header, age, gender,
    vitals_header, bp_systolic, bp_diastolic, chol_total, chol_hdl, bmi, glucose,
    lifestyle_header, exercise, smoking, alcohol, stress, sleep,
    symptoms_header, symptoms,
    widgets.HBox([predict_button], layout=widgets.Layout(justify_content='center'))
])

display(widgets.VBox([input_box, tab, output]))

# Enhanced prediction function with symptoms integration
def on_predict_button_clicked(b):
    with output:
        clear_output()
        
        # Calculate derived metrics
        bp_mean = (bp_systolic.value + 2 * bp_diastolic.value) / 3
        chol_ratio = chol_total.value / chol_hdl.value if chol_hdl.value > 0 else 5
        
        # Prepare input data for model
        input_data = np.array([[
            age.value,
            bp_mean,  # Using mean arterial pressure
            chol_total.value,
            bmi.value,
            glucose.value,
            exercise.value,
            1 if smoking.value == 2 else 0,  # Current smoker
            alcohol.value,
            stress.value,
            sleep.value
        ]])
        
        # Make predictions
        predictions = model.predict(input_data)[0]
        probabilities = model.predict_proba(input_data)
        
        # Get condition names and probabilities
        condition_probabilities = []
        for i, target in enumerate(targets):
            condition_probabilities.append({
                'condition': target,
                'name': diet_plans[target]['name'],
                'predicted': predictions[i],
                'probability': probabilities[i][0][1]  # Probability of positive class
            })
        
        # Sort by probability descending
        condition_probabilities.sort(key=lambda x: x['probability'], reverse=True)
        
        # Determine active conditions (using 30% probability threshold)
        active_conditions = [c['condition'] for c in condition_probabilities if c['probability'] > 0.3]
        
        # Get combined recommendations
        combined_rec = get_combined_recommendations(predictions, active_conditions)
        
        # Display Health Summary
        with health_summary:
            clear_output()
            if active_conditions:
                display(HTML(
                    f"<h2 style='color: #e74c3c;'>⚠️ Potential Health Conditions Detected</h2>"
                    f"<h3>{combined_rec['name']}</h3>"
                ))
                
                # Display top 3 risk factors
                risk_factors = []
                if age.value > 50: risk_factors.append(f"Age ({age.value})")
                if bp_systolic.value > 130 or bp_diastolic.value > 80: 
                    risk_factors.append(f"Blood Pressure ({bp_systolic.value}/{bp_diastolic.value})")
                if chol_ratio > 4: risk_factors.append(f"Cholesterol Ratio ({chol_ratio:.1f})")
                if bmi.value > 25: risk_factors.append(f"BMI ({bmi.value})")
                if glucose.value > 100: risk_factors.append(f"Glucose ({glucose.value})")
                if smoking.value == 2: risk_factors.append("Current Smoker")
                if alcohol.value == 2: risk_factors.append("Heavy Alcohol Use")
                if stress.value == 2: risk_factors.append("High Stress")
                if sleep.value < 6: risk_factors.append(f"Inadequate Sleep ({sleep.value} hrs)")
                
                if risk_factors:
                    display(HTML("<h4>🔍 Key Risk Factors:</h4>"))
                    display(HTML("<ul style='column-count: 2;'>" + 
                                "".join([f"<li>{factor}</li>" for factor in risk_factors[:6]]) + 
                                "</ul>"))
            else:
                display(HTML(
                    "<h2 style='color: #2ecc71;'>✅ No Significant Health Conditions Detected</h2>"
                    "<p>Your health metrics appear to be within normal ranges. Maintain your healthy lifestyle!</p>"
                ))
        
        # Display Detailed Reports
        with detailed_reports:
            clear_output()
            display(HTML("<h2>Detailed Health Report</h2>"))
            
            # Create a table with condition probabilities
            table_html = """
            <table border='1' style='border-collapse: collapse; width: 100%;'>
                <tr style='background-color: #f2f2f2;'>
                    <th>Condition</th>
                    <th>Status</th>
                    <th>Probability</th>
                    <th>Risk Level</th>
                </tr>
            """
            
            for cond in condition_probabilities:
                if cond['probability'] > 0.1:  # Only show conditions with >10% probability
                    risk_level = "Low"
                    if cond['probability'] > 0.7: risk_level = "High"
                    elif cond['probability'] > 0.4: risk_level = "Moderate"
                    
                    status = "⚠️ Detected" if cond['predicted'] == 1 else "✅ Normal"
                    color = "#e74c3c" if cond['predicted'] == 1 else "#2ecc71"
                    
                    table_html += f"""
                    <tr>
                        <td>{cond['name']}</td>
                        <td style='color: {color};'>{status}</td>
                        <td>{cond['probability']*100:.1f}%</td>
                        <td>{risk_level}</td>
                    </tr>
                    """
            
            table_html += "</table>"
            display(HTML(table_html))
            
            # Show relevant symptoms
            if symptoms.value:
                display(HTML(
                    f"<h4>🗒️ Reported Symptoms:</h4>"
                    f"<ul style='column-count: 2;'>" + 
                    "".join([f"<li>{symptom}</li>" for symptom in symptoms.value]) + 
                    "</ul>"
                ))
        
        # Display Diet Plan
        with diet_plan:
            clear_output()
            display(HTML(f"<h2>🍎 Personalized Diet Plan</h2>"))
            
            if active_conditions:
                display(HTML(
                    f"<h3 style='color: #3498db;'>Dietary Recommendations for {combined_rec['name']}</h3>"
                    "<h4>📋 Key Recommendations:</h4>"
                    "<ul>" + 
                    "".join([f"<li>{rec}</li>" for rec in combined_rec['recommendations'][:5]]) + 
                    "</ul>"
                    
                    "<h4 style='color: #2ecc71;'>✅ Beneficial Foods:</h4>"
                    "<div style='column-count: 2;'>" + 
                    "".join([f"<li>{food}</li>" for food in combined_rec['beneficial_foods'][:10]]) + 
                    "</div>"
                    
                    "<h4 style='color: #e74c3c;'>🚫 Foods to Limit/Avoid:</h4>"
                    "<div style='column-count: 2;'>" + 
                    "".join([f"<li>{food}</li>" for food in combined_rec['foods_to_avoid'][:10]]) + 
                    "</div>"
                ))
            else:
                display(HTML(
                    "<h3>General Healthy Eating Guidelines</h3>"
                    "<p>" + "<br>".join(diet_plans['general_health']['recommendations']) + "</p>"
                ))
        
        # Display Lifestyle Plan
        with lifestyle_plan:
            clear_output()
            display(HTML("<h2>🌿 Lifestyle Recommendations</h2>"))
            
            lifestyle_recs = []
            
            # Exercise recommendations
            if exercise.value < 3:
                lifestyle_recs.append("""
                <li><b>Exercise:</b> Increase physical activity to at least 150 minutes of 
                moderate exercise per week (e.g., 30 minutes 5 days/week). Consider:
                <ul>
                    <li>Brisk walking</li>
                    <li>Swimming</li>
                    <li>Cycling</li>
                </ul>
                </li>
                """)
            else:
                lifestyle_recs.append("""
                <li><b>Exercise:</b> You're meeting the recommended exercise guidelines. 
                Maintain your current routine and consider adding strength training 2x/week.</li>
                """)
            
            # Stress management
            if stress.value > 0:
                lifestyle_recs.append("""
                <li><b>Stress Management:</b> Consider incorporating stress-reduction techniques:
                <ul>
                    <li>Mindfulness meditation (10-15 minutes daily)</li>
                    <li>Deep breathing exercises</li>
                    <li>Yoga or tai chi</li>
                    <li>Regular social connections</li>
                </ul>
                </li>
                """)
            
            # Sleep recommendations
            if sleep.value < 7:
                lifestyle_recs.append(f"""
                <li><b>Sleep:</b> Aim for 7-9 hours of quality sleep nightly. Your current 
                {sleep.value} hours may be insufficient. Try:
                <ul>
                    <li>Consistent sleep schedule</li>
                    <li>Screen-free time before bed</li>
                    <li>Cool, dark sleeping environment</li>
                </ul>
                </li>
                """)
            
            # Smoking cessation
            if smoking.value == 2:
                lifestyle_recs.append("""
                <li><b>Smoking Cessation:</b> Quitting smoking is the single most important 
                thing you can do for your health. Consider:
                <ul>
                    <li>Nicotine replacement therapy</li>
                    <li>Prescription medications</li>
                    <li>Behavioral counseling</li>
                    <li>Support groups</li>
                </ul>
                </li>
                """)
            
            display(HTML("<ul>" + "".join(lifestyle_recs) + "</ul>"))
        
        # Display Visualization
        with visualization:
            clear_output()
            
            # Create radar chart for health metrics
            metrics = ['BP', 'Cholesterol', 'BMI', 'Glucose', 'Exercise', 'Sleep']
            values = [
                min((bp_systolic.value - 80) / (140 - 80) * 100, 100),
                min((chol_total.value - 100) / (240 - 100) * 100, 100),
                min((bmi.value - 18) / (30 - 18) * 100, 100),
                min((glucose.value - 70) / (126 - 70) * 100, 100),
                (exercise.value / 5) * 100,
                (sleep.value / 9) * 100
            ]
            
            fig = go.Figure()
            
            fig.add_trace(go.Scatterpolar(
                r=values,
                theta=metrics,
                fill='toself',
                name='Your Metrics',
                line_color='#3498db'
            ))
            
            # Add ideal ranges
            ideal_values = [50, 50, 50, 50, 100, 100]  # Ideal is middle for most, high for exercise/sleep
            fig.add_trace(go.Scatterpolar(
                r=ideal_values,
                theta=metrics,
                name='Ideal Range',
                line_color='#2ecc71'
            ))
            
            fig.update_layout(
                polar=dict(
                    radialaxis=dict(
                        visible=True,
                        range=[0, 100]
                    )),
                showlegend=True,
                title="Health Metrics Radar Chart"
            )
            
            fig.show()
            
            # Create bar chart for top conditions
            top_conditions = [c for c in condition_probabilities if c['probability'] > 0.1][:5]
            if top_conditions:
                fig2 = px.bar(
                    x=[c['name'] for c in top_conditions],
                    y=[c['probability'] * 100 for c in top_conditions],
                    title="Top Health Conditions by Probability",
                    labels={'x': 'Condition', 'y': 'Probability (%)'},
                    color=[c['probability'] for c in top_conditions],
                    color_continuous_scale=px.colors.sequential.Reds
                )
                fig2.update_layout(yaxis_range=[0, 100])
                fig2.show()

# Link button to function
predict_button.on_click(on_predict_button_clicked)

In [15]:
pip install voila pandas numpy scikit-learn plotly ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!voila OPEN-HEALTH-ADVANCED.ipynb --enable_nbextensions=True

In [ ]:
!voila OPEN-HEALTH-ADVANCED.ipynb --port=8080 --no-browser --enable_nbextensions=True --Voila.ip=0.0.0.0